In [53]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Conv2D, Conv1D, MaxPooling2D, Dropout, Conv2DTranspose
from tensorflow.keras.layers import UpSampling2D, add, Cropping2D, ReLU, BatchNormalization, Lambda, PReLU
from tensorflow.keras.layers import Concatenate, Reshape, MaxPooling1D, Cropping1D, ZeroPadding1D, Flatten
from tensorflow.keras.layers import AveragePooling2D, LSTM, RepeatVector, TimeDistributed, ZeroPadding2D
from tensorflow.keras.layers import LeakyReLU, Layer, Activation, multiply
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.constraints import Constraint
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import Sequence, OrderedEnqueuer
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras import activations, layers
from tensorflow.keras.backend import expand_dims, mean, clip, set_image_data_format

import os
import re
import datetime
from scipy import ndimage, misc
from skimage.transform import resize, rescale
from matplotlib import pyplot
import numpy as np
import matplotlib.pyplot as plt
import random

devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(devices[0], True)
#tf.keras.backend.set_floatx('float16')

In [54]:
class GatedCNN(object):
    ''' Convolution layer with gated activation unit. '''

    def __init__(self, nb_filters, stack_name, v_map=None, h=None, crop_right=False, **kwargs):
        '''
        Args:
            nb_filters (int)         : Number of the filters (feature maps)
            stack_name (str)		: 'vertical' or 'horizontal'
            v_map (numpy.ndarray)   : Vertical maps if feeding into horizontal stack. (default:None)
            h (numpy.ndarray)       : Latent vector to model the conditional distribution p(x|h) (default:None)
            crop_right (bool)       : if True, crop rightmost of the feature maps (mask A, introduced in [https://arxiv.org/abs/1601.06759] )
        '''
        self.nb_filters = nb_filters
        self.stack_name = stack_name
        self.v_map = v_map
        self.h = h
        self.crop_right = crop_right

    @staticmethod
    def _crop_right(x):
        x_shape = K.int_shape(x)
        return x[:,:,:x_shape[2]-1,:]

    def __call__(self, xW, layer_idx):
        '''calculate gated activation maps given input maps '''
        if self.stack_name == 'vertical':
            stack_tag = 'v'
        elif self.stack_name == 'horizontal':
            stack_tag = 'h'

        if self.crop_right:
            xW = Lambda(self._crop_right, name='h_crop_right_'+str(layer_idx))(xW)

        if self.v_map is not None:
            #xW = merge([xW, self.v_map], mode='sum', name='h_merge_v_'+str(layer_idx))
            xW = add([xW, self.v_map], name='h_merge_v_'+str(layer_idx))
        
        if self.h is not None:
            hV = Dense(output_dim=2*self.nb_filters, name=stack_tag+'_dense_latent_'+str(layer_idx))(self.h)
            hV = Reshape((1, 1, 2*self.nb_filters), name=stack_tag+'_reshape_latent_'+str(layer_idx))(hV)
            #xW = merge([xW, hV], mode=lambda x: x[0]+x[1])
            xW = Lambda(lambda x: x[0]+x[1], name=stack_tag+'_merge_latent_'+str(layer_idx))([xW,hV])

        xW_f = Lambda(lambda x: x[:,:,:,:self.nb_filters], name=stack_tag+'_Wf_'+str(layer_idx))(xW)
        xW_g = Lambda(lambda x: x[:,:,:,self.nb_filters:], name=stack_tag+'_Wg_'+str(layer_idx))(xW)

        xW_f = Lambda(lambda x: K.tanh(x), name=stack_tag+'_tanh_'+str(layer_idx))(xW_f)
        xW_g = Lambda(lambda x: K.sigmoid(x), name=stack_tag+'_sigmoid_'+str(layer_idx))(xW_g)

        #res = merge([xW_f, xW_g], mode='mul', name=stack_tag+'_merge_gate_'+str(layer_idx))
        res = multiply([xW_f, xW_g], name=stack_tag+'_merge_gate_'+str(layer_idx))
        #print(type(res), K.int_shape(res), hasattr(res, '_keras_history'))
        return res

#############################################################################################################

class PixelCNN(object):
    ''' Keras implementation of (conditional) Gated PixelCNN model '''
    def __init__(self, input_size, nb_channels=3, conditional=False, latent_dim=10,
        nb_pixelcnn_layers=13, nb_filters=128, filter_size_1st=(7,7), filter_size=(3,3),
        optimizer='adadelta', es_patience=100, save_root='/tmp/pixelcnn', save_best_only=False,
        **kwargs):
        '''
        Args:
            input_size ((int,int))      : (height, width) pixels of input images
            nb_channels (int)           : Number of channels for input images. (1 for grayscale images, 3                                               for color images)
            conditional (bool)          : if True, use latent vector to model the conditional distribution p                                            (x|h) (default:False)
            latent_dim (int)            : (if conditional==True,) Dimensions for latent vector.
            nb_pixelcnn_layers (int)    : Number of layers (except last two ReLu layers). (default:13)
            nb_filters (int)            : Number of filters (feature maps) for each layer. (default:128)
            filter_size_1st ((int, int)): Kernel size for the first layer. (default: (7,7))
            filter_size ((int, int))    : Kernel size for the subsequent layers. (default: (3,3))
            optimizer (str)             : SGD optimizer (default: 'adadelta')
            es_patience (int)           : Number of epochs with no improvement after which training will be                                             stopped (EarlyStopping)
            save_root (str)             : Root directory to which {trained model file, parameter.txt,                                                   tensorboard log file} are saved
            save_best_only (bool)       : if True, the latest best model will not be overwritten (default:                                              False)
        '''
        #K.set_image_dim_ordering('tf')

        self.input_size = input_size
        self.conditional = conditional
        self.latent_dim = latent_dim
        self.nb_pixelcnn_layers = nb_pixelcnn_layers
        self.nb_filters = nb_filters
        self.filter_size_1st = filter_size_1st
        self.filter_size = filter_size
        self.nb_channels = nb_channels
        if self.nb_channels == 1:
            self.loss = 'binary_crossentropy'
        elif self.nb_channels == 3:
            self.loss = 'categorical_crossentropy'
        self.optimizer = optimizer
        self.es_patience = es_patience
        self.save_best_only = save_best_only

        tensorboard_dir = os.path.join(save_root, 'pixelcnn-tensorboard')
        checkpoint_path = os.path.join(save_root, 'pixelcnn-weights.{epoch:02d}-{val_loss:.4f}.hdf5')
        self.tensorboard = TensorBoard(log_dir=tensorboard_dir)
        ### "save_weights_only=False" causes error when exporting model architecture. (json or yaml)
        self.checkpointer = ModelCheckpoint(filepath=checkpoint_path, verbose=1,
                            save_weights_only=True, save_best_only=save_best_only)
        self.earlystopping = EarlyStopping(monitor='val_loss', patience=es_patience, verbose=0, mode='auto')
    

    def _masked_conv(self, x, filter_size, stack_name, layer_idx, mask_type='B'):
        if stack_name == 'vertical':
            res = ZeroPadding2D(
                padding=((filter_size[0]//2, 0), (filter_size[1]//2, filter_size[1]//2)),
                name='v_pad_'+str(layer_idx))(x)
            res = Conv2D(2*self.nb_filters, filter_size[0]//2+1, filter_size[1],
                padding='valid', name='v_conv_'+str(layer_idx))(res)

        elif stack_name == 'horizontal':
            res = ZeroPadding2D(padding=((0, 0), (filter_size[1]//2, 0)), name='h_pad_'+str(layer_idx))(x)
            if mask_type == 'A':
                res = Conv2D(2*self.nb_filters, 1, filter_size[1]//2,
                    padding='valid', name='h_conv_'+str(layer_idx))(res)
            else:
                res = Conv2D(2*self.nb_filters, 1, filter_size[1]//2+1,
                    padding='valid', name='h_conv_'+str(layer_idx))(res)

        return res


    @staticmethod
    def _shift_down(x):
        x_shape = K.int_shape(x)
        x = ZeroPadding2D(padding=((1,0),(0,0)))(x)
        x = Lambda(lambda x: x[:,:x_shape[1],:,:])(x)
        return x

    def _feed_v_map(self, x, layer_idx):
        ### shifting down feature maps
        x = Lambda(self._shift_down, name='v_shift_down'+str(layer_idx))(x)
        x = Conv2D(2*self.nb_filters, 1, 1, padding='valid', name='v_1x1_conv_'+str(layer_idx))(x)
        return x


    def _build_layers(self, x, h=None):
        ''' Whole architecture of (conditional) Gated PixelCNN model '''
        # set latent vector
        self.h = h

        # first PixelCNN layer
        ### (kxk) masked convolution can be achieved by (k//2+1, k) convolution and padding.
        v_masked_map = self._masked_conv(x, self.filter_size_1st, 'vertical', 0)
        ### (i-1)-th vertical activation maps into the i-th hirizontal stack.
        # (if i==0, vertical activation maps == input images)
        v_feed_map = self._feed_v_map(v_masked_map, 0)
        v_stack_out = GatedCNN(self.nb_filters, 'vertical', v_map=None, h=self.h)(v_masked_map, 0)
        ### (1xk) masked convolution can be achieved by (1 x k//2+1) convolution and padding.
        h_masked_map = self._masked_conv(x, self.filter_size_1st, 'horizontal', 0, 'A')
        ### Mask A is applied to the first layer (achieved by cropping), and v_feed_maps are merged. 
        h_stack_out = GatedCNN(self.nb_filters, 'horizontal', v_map=v_feed_map,
            h=self.h, crop_right=True)(h_masked_map, 0)
        ### not residual connection in the first layer.
        h_stack_out = Conv2D(self.nb_filters, 1, 1, padding='valid',
            name='h_1x1_conv_0')(h_stack_out)

        # subsequent PixelCNN layers
        for i in range(1, self.nb_pixelcnn_layers):
            v_masked_map = self._masked_conv(v_stack_out, self.filter_size, 'vertical', i)
            v_feed_map = self._feed_v_map(v_masked_map, i)
            v_stack_out = GatedCNN(self.nb_filters, 'vertical', v_map=None, h=self.h)(v_masked_map, i)
            ### for residual connection
            h_stack_out_prev = h_stack_out
            h_masked_map = self._masked_conv(h_stack_out, self.filter_size, 'horizontal', i)
            ### Mask B is applied to the subsequent layers.
            h_stack_out = GatedCNN(self.nb_filters, 'horizontal', v_map=v_feed_map,
                h=self.h)(h_masked_map, i)
            h_stack_out = Conv2D(self.nb_filters, 1, 1, padding='valid',
                name='h_1x1_conv_'+str(i))(h_stack_out)
            ### residual connection
            #h_stack_out = merge([h_stack_out, h_stack_out_prev], mode='sum', name='h_residual_'+str(i))
            h_stack_out = add([h_stack_out, h_stack_out_prev], name='h_residual_'+str(i))

        # (1x1) convolution layers (2 layers)
        for i in range(2):
            h_stack_out = Conv2D(self.nb_filters, 1, 1, activation='relu',
                padding='valid', name='penultimate_convs'+str(i))(h_stack_out)
        
        # Softmax layer (256-way for each RGB color (natural image) or sigmoid for each pixel (MNIST))
        if self.nb_channels == 1:
            res = Conv2D(1, 1, 1, activation='sigmoid', padding='valid')(h_stack_out)
            #res = Reshape((self.input_size[0]*self.input_size[1], 1))(res)
            return res
        elif self.nb_channels == 3:
            ### 256-way * 3(channels) = 768
            res = Conv2D(768, nb_row=1, nb_col=1, padding='valid')(h_stack_out)
            res = Reshape((self.input_size[0] * self.input_size[1] * 3, 256))(res)
            return Activation('softmax')(res)
        else:
            return None


    def build_model(self):
        ''' build conditional PixelCNN model '''
        if self.nb_channels == 1:
            input_img = Input(shape=(self.input_size[0], self.input_size[1], 1), name='grayscale_image')
        elif self.nb_channels == 3:
            input_img = Input(shape=(self.input_size[0], self.input_size[1], 3), name='color_image')

        if self.conditional:
            latent_vector = Input(shape=(self.latent_dim,), name='latent_vector')
            predicted = self._build_layers(input_img, latent_vector)
            self.model = Model(input=[input_img, latent_vector], output=predicted)
        else:
            predicted = self._build_layers(input_img)
            self.model = Model(input_img, predicted)

        self.model.compile(optimizer=self.optimizer, loss=self.loss)
    

    def fit(self, x, y, batch_size, nb_epoch, validation_data=None, shuffle=True):
        ''' call fit function
        Args:
            x (np.ndarray or [np.ndarray, np.ndarray])  : Input data for training
            y (np.ndarray)                              : Label data for training 
            samples_per_epoch (int)                     : Number of data for each epoch
            nb_epoch (int)                              : Number of epoches
            validation_data ((np.ndarray, np.ndarray))  : Validation data
            nb_val_samples (int)                        : Number of data yielded by validation generator
            shuffle (bool)                              : if True, shuffled randomly
        '''
        self.model.fit(x=x, y=y, batch_size=batch_size, nb_epoch=nb_epoch,
            callbacks=[self.tensorboard, self.checkpointer, self.earlystopping],
            validation_data=validation_data, shuffle=shuffle)

    def fit_generator(self, train_generator, samples_per_epoch, nb_epoch,
        validation_data=None, nb_val_samples=10000):
        ''' call fit_generator function
        Args:
            train_generator (object)        : image generator built by "build_generator" function
            samples_per_epoch (int)         : Number of data for each epoch
            nb_epoch (int)                  : Number of epoches
            validation_data (object/array)  : generator object or numpy.ndarray
            nb_val_samples (int)            : Number of data yielded by validation generator
        '''
        self.model.fit_generator(generator=train_generator, samples_per_epoch=samples_per_epoch,
            nb_epoch=nb_epoch, callbacks=[self.tensorboard, self.checkpointer, self.earlystopping],
            validation_data=validation_data, nb_val_samples=nb_val_samples)


    def load_model(self, checkpoint_file):
        ''' restore model from checkpoint file (.hdf5) '''
        self.model = load_model(checkpoint_file)

    def export_to_json(self, save_root):
        ''' export model architecture config to json file '''
        with open(os.path.join(save_root, 'pixelcnn_model.json'), 'w') as f:
            f.write(self.model.to_json())

    def export_to_yaml(self, save_root):
        ''' export model architecture config to yaml file '''
        with open(os.path.join(save_root, 'pixelcnn_model.yml'), 'w') as f:
            f.write(self.model.to_yaml())


    @classmethod
    def predict(self, x, batch_size):
        ''' generate image pixel by pixel
        Args:
            x or [x,h] (x,h: numpy.ndarray : x = input image, h = latent vector
        Returns:
            predict (numpy.ndarray)        : generated image
        '''
        return self.model.predict(x, batch_size)


    def print_train_parameters(self, save_root):
        ''' print parameter list file '''
        print('\n########## PixelCNN options ##########')
        print('input_size\t: %s' % (self.input_size,))
        print('nb_pixelcnn_layers: %s' % self.nb_pixelcnn_layers)
        print('nb_filters\t: %s' % self.nb_filters)
        print('filter_size_1st\t: %s' % (self.filter_size_1st,))
        print('filter_size\t: %s' % (self.filter_size,))
        print('conditional\t: %s' % self.conditional)
        print('nb_channels\t: %s' % self.nb_channels)
        print('optimizer\t: %s' % self.optimizer)
        print('loss\t\t: %s' % self.loss)
        print('es_patience\t: %s' % self.es_patience)
        print('save_root\t: %s' % save_root)
        print('save_best_only\t: %s' % self.save_best_only)
        print('\n')

    def export_train_parameters(self, save_root):
        ''' export parameter list file '''
        with open(os.path.join(save_root, 'parameters.txt'), 'w') as txt_file:
            txt_file.write('########## PixelCNN options ##########\n')
            txt_file.write('input_size\t: %s\n' % (self.input_size,))
            txt_file.write('nb_pixelcnn_layers: %s\n' % self.nb_pixelcnn_layers)
            txt_file.write('nb_filters\t: %s\n' % self.nb_filters)
            txt_file.write('filter_size_1st\t: %s\n' % (self.filter_size_1st,))
            txt_file.write('filter_size\t: %s\n' % (self.filter_size,))
            txt_file.write('conditional\t: %s\n' % self.conditional)
            txt_file.write('nb_channels\t: %s\n' % self.nb_channels)
            txt_file.write('optimizer\t: %s\n' % self.optimizer)
            txt_file.write('loss\t\t: %s\n' % self.loss)
            txt_file.write('es_patience\t: %s\n' % self.es_patience)
            txt_file.write('save_root\t: %s\n' % save_root)
            txt_file.write('save_best_only\t: %s\n' % self.save_best_only)
            txt_file.write('\n')

In [55]:
import os
from skimage.io import imread
#import matplotlib
#matplotlib.use('Agg')
import matplotlib.pyplot as plt
import random
import numpy as np


class Utils(object):

    @staticmethod
    def image2kerasarray(img):
        ''' reshape image array 
        Args:
            img (numpy.ndarray)     : common image array ((nb_images,)height,width(,3))
        Returns:
            array (numpy.ndarray)   : keras array ((nb_images,)height,width,channels)
        '''
        array = img.astype('float32') / 255.
        if array.shape[-1] != 3: ### (num_images, height, width) ###
            array = np.expand_dims(array, axis=-1)

        return array

    @staticmethod
    def image2labelmap(img):
        ''' convert color image to label map
        Args:
            img (numpy.ndarray)     : common color image array (height,width,3)
        Returns:
            label_map (np.ndarray)	: label map (height*width*3,256)
        '''
        height, width, _  = img.shape
        label_map = np.zeros([height*width*3, 256])
        for h in range(height):
            for w in range(width):
                for c in range(3):
                    label_map[3*width*h + 3*w + c, img[h][w][c]] = 1

        return label_map

    @classmethod
    def load_mnist_datasets(cls, conditional=False):
        ''' load mnist dataset
        Args:
            conditional (bool)	: if True, return image arrays and class label vectors. if False, return                                        only image arrays.
        Returns:
            ([X_train, h_train], Y_train), ([X_validation, h_validation], Y_validation)	: if conditional ==                                                                                                         True
            (X_train, Y_train), (X_validation, Y_validation)							: if conditional ==                                                                                                         False
        *** Loading {'cifar10', 'cifar100'} dataset causes MemoryError due to the softmax layer. ***
        '''
        from keras.datasets import mnist
        (X_train, h_train), (X_validation, h_validation) = mnist.load_data()
        nb_classes = 10

        from keras.utils import np_utils

        X_train = cls.image2kerasarray(X_train)
        X_validation = cls.image2kerasarray(X_validation)

        ### In case single channel, Y = X ###
        X_train = cls.binarize_array(X_train)
        X_validation = cls.binarize_array(X_validation)
        Y_train = X_train
        Y_validation = X_validation

        ### If conditional == True, use class labels as latent vector ###
        if conditional:
            h_train = np_utils.to_categorical(h_train, nb_classes)
            h_validation = np_utils.to_categorical(h_validation, nb_classes)


        if conditional:
            return (([X_train, h_train], Y_train), ([X_validation, h_validation], Y_validation))
        else:
            return ((X_train, Y_train), (X_validation, Y_validation))

    @classmethod
    def build_data_generator_from_keras_datasets(cls, dataset_name, X, H=None, batch_size=100):
        ''' kerasarray generator without keras ImageDataGenerator
            Args:
                dataset_name (str)	: {'mnist', 'cifar10', 'cifar100'}
                X (numpy.ndarray)	: Image arrays. (nb_images, height, width (, 3))
                H (numpy.ndarray)	: Latent vectors (nb_images, nb_classes)
                batch_size (int)	: minibatch size that generator yields at once
        '''
        from keras.utils import np_utils

        if dataset_name == 'mnist':
            target_size = (28, 28)
            nb_classes = 10
        elif dataset_name == 'cifar10':
            target_size = (32, 32)
            nb_classes = 10
        elif dataset_name == 'cifar100':
            target_size = (32, 32)
            nb_classes = 100

        while 1:
            if dataset_name == 'mnist':
                x = np.zeros((batch_size, target_size[0], target_size[1], 1))
                y = np.zeros((batch_size, target_size[0], target_size[1], 1))
            else:
                x = np.zeros((batch_size, target_size[0], target_size[1], 3))
                y = np.zeros((batch_size, target_size[0]*target_size[1]*3, 256))
            if H is not None:
                h = np.zeros((batch_size, nb_classes))
                H = np_utils.to_categorical(H, nb_classes)

            batch_idx = 0
            shuffled_index = list(range(len(X)))
            random.shuffle(shuffled_index)

            for i in shuffled_index:
                if dataset_name == 'mnist':
                    binarized_X = cls.binarize_array(cls.image2kerasarray(X[i]))
                    x[batch_idx % batch_size] = binarized_X
                    y[batch_idx % batch_size] = binarized_X
                else:
                    x[batch_idx % batch_size] = cls.image2kerasarray(X[i])
                    y[batch_idx % batch_size] = cls.image2labelmap(X[i])
                if h is not None:
                    h[batch_idx % batch_size] = H[i]
                batch_idx += 1
                if (batch_idx % batch_size) == 0:
                    if h is not None:
                        yield ([x, h], y)
                    else:
                        yield (x, y)

    @staticmethod
    def read_image(path):
        ''' read image from filepath
        Args:
            path (str)              : filepath (/path/to/img.jpg)
        Returns:
            image (numpy.ndarray)   : np.array of shepe (height, width, channels)
        '''
        img = imread(path)
        return img

    @classmethod
    def build_data_generator_from_directory(cls, target_size, data_paths, batch_size=100):
        ''' kerasarray generator without keras ImageDataGenerator
            Args:
                target_size (int,int)       : (height, width) pixels of image
                data_paths (list(str))      : ["/path/to/image001.jpg", "/path/to/image002.jpg", ...]
                batch_size (int)            : minibatch size that generator yields at once
        '''
        while 1:
            x = np.zeros((batch_size, target_size[0], target_size[1], 3))
            y = np.zeros((batch_size, target_size[0]*target_size[1]*3, 256))
            batch_idx = 0
            shuffled_index = list(range(len(data_paths)))
            random.shuffle(shuffled_index)

            for i in shuffled_index:
                x[batch_idx % batch_size] = cls.image2kerasarray(cls.read_image(data_paths[i]))
                y[batch_idx % batch_size] = cls.image2labelmap(cls.read_image(data_paths[i]))
                batch_idx += 1
                if (batch_idx % batch_size) == 0:
                    yield (x, y)
    
    @staticmethod
    def binarize_val(pred):
       ''' probability -> binarized value '''
       return (np.random.uniform(size=1) < pred).astype(np.float32)

    @staticmethod
    def binarize_array(array):
        ''' scaled image (value range:0-1) -> binarized array '''
        return (np.random.uniform(size=array.shape) < array).astype(np.float32)
    
    @staticmethod
    def sample(preds, temperature=1.0):
        # helper function to sample an index from a probability array
        # copied from https://github.com/fchollet/keras/blob/master/examples/lstm_text_generation.py
        preds = np.asarray(preds).astype('float32')
        preds = np.log(preds) / temperature
        exp_preds = np.exp(preds)
        preds = exp_preds / np.sum(exp_preds)
        probas = np.random.multinomial(1, preds, 1)

        return np.argmax(probas).astype('float32') / 255.

    @staticmethod
    def save_generated_image(img, filename="img.jpg", save_path='/tmp/pixelcnn/results'):
        ''' save predicted image
        Args:
            img (numpy.array)       : Image array
            filename (str)          : Save image to save_path/filename
            save_root (str)         : Save image to save_path/filename
        '''
        if not os.path.exists(save_path):
            os.makedirs(save_path)

        fig = plt.figure(figsize=(15, 15))
        ax = plt.subplot(1, 1, 1)
        ax.imshow(img)
        fig.savefig(os.path.join(save_path, filename))
        plt.close(fig)


    @staticmethod
    def save_generated_images(imgs, cols, filename='img.jpg', save_path='/tmp/pixelcnn/results'):
        ''' save predicted images
        Args:
            imgs (numpy.array)		: Image arrays
            cols (int)				: number of columns for visualizing images
            filename (str)			: Save image to save_path/filename
            save_root (str)			: Save image to save_path/filename
        '''
        if not os.path.exists(save_path):
            os.makedirs(save_path)

        fig = plt.figure()
        rows = len(imgs) // cols + 1

        for j in range(len(imgs)):
            ax = fig.add_subplot(cols, rows, j+1)
            ax.matshow(imgs[j], cmap=matplotlib.cm.binary)
            plt.xticks(np.array([]))
            plt.yticks(np.array([]))
        plt.tight_layout()

        fig.savefig(os.path.join(save_path, filename))
        plt.close(fig)

In [57]:
pixelcnn = PixelCNN((32, 32))
pixelcnn.build_model()
pixelcnn.model.summary()

ValueError: Operands could not be broadcast together with shapes (11, 11, 256) (5, 5, 256)